In [152]:
import requests
from key import fred_key
import pandas as pd

base_url = "https://api.stlouisfed.org/fred/"
obs_endpoint = "series/observations"

start_date = "1950-01-01"
end_date = "2023-12-31"



d_gdp = {
    'name': 'GDP',
    'series_id': "GDPC1",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_cnd = {
    'name': 'CND',
    'series_id': "PCEND",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_cd = {
    'name': 'CD',
    'series_id': "PCEDG",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

# full-time and part-time employees
d_h = {
    'name': 'H',
    'series_id': "B4701C0A222NBEA",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}



# excluding farm employees
d_l = {
    'name': 'L',
    'series_id': "PAYEMS",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

d_AveW = {
    'name': 'AveW',
    'series_id': "AHETPI",
    'api_key': fred_key,
    'file_type': 'json',
    'observation_start': start_date,
    'observation_end': end_date,
    'frequency': 'a',
    'units': 'lin'
}

request_parameters = [d_gdp, d_cd, d_cnd, d_h, d_l, d_AveW]




In [153]:
def fetch_data(parameter):
    response = requests.get(base_url + obs_endpoint, params=parameter)
    if response.status_code == 200:
        res_data = response.json()
        obs_data = pd.DataFrame(res_data['observations'])
        obs_data['date'] = pd.to_datetime(obs_data['date'])
        obs_data.set_index('date', inplace=True)
        obs_data['value'] = obs_data['value'].astype(float)
        obs_data = obs_data.drop(['realtime_start', 'realtime_end'], axis=1)
        obs_data.rename(columns={'value': parameter['name']}, inplace=True)
        return obs_data
    else:
        print('Failed to retrieve data. Status code:', response.status_code)
        


dataframe = pd.DataFrame()
# concatenate the data
for parameter in request_parameters:
    df = fetch_data(parameter)
    if dataframe.empty:
        dataframe = df
    else:
        dataframe = pd.concat([dataframe, df], axis=1)

# Display the merged dataframe
dataframe

,GDP,CD,CND,H,L,AveW
date,,,,,,
1950-01-01,2458.532,NaN,NaN,100064.0,45282.0,NaN
1951-01-01,2656.320,NaN,NaN,108525.0,47926.0,NaN
1952-01-01,2764.803,NaN,NaN,110757.0,48903.0,NaN
1953-01-01,2894.412,NaN,NaN,112184.0,50304.0,NaN
1954-01-01,2877.708,NaN,NaN,107712.0,49087.0,NaN
...,...,...,...,...,...,...
2019-01-01,20692.087,1522.7,3006.5,257352.0,150906.0,23.51
2020-01-01,20234.074,1628.9,3084.2,242613.0,142165.0,24.69
2021-01-01,21407.693,2006.4,3500.2,251215.0,146276.0,25.91


In [156]:
# adjust the dataset

# add productivity
dataframe['GDP/L'] = (dataframe['GDP']/dataframe['L'])  # annual average income
dataframe['AveH'] = dataframe['H']/dataframe['L']*1000
dataset1 = dataframe.dropna()
  # data from 1964 to 2022
dataset1.to_csv("data1.csv",index=False)


In [ ]:
# replicate the code